In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from core.load_data import load_data, initalize_result_df

from core.preprocessing import decoding_numbers, train_val_test_split
from core.preprocessing import list_resampler, all_columns

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from imblearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
#ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Load data and transform the numbers
path = '../data/raw/2011 Census Microdata Teaching File.csv'
df = pd.read_csv(path, skiprows= 1)
df_decoded = decoding_numbers(df)


### Train Test Validation Split


In [3]:
features_train, target_train, features_val, target_val, features_test, target_test = train_val_test_split(df_decoded.iloc[:,:-1], df_decoded.iloc[:,-1])

Resample the data to speed up the calculations.

In [4]:
features_train_resampled = features_train.sample(n=10000, axis = 0, random_state=42)
target_train_resampled = target_train.sample(n=10000, axis = 0, random_state=42)

Initialize dataframe for results.

In [5]:
initalize_result_df()
path_results = '../data/processed/Comparison_models.csv'
results = pd.read_csv(path_results)


## Functions

Function which builds pipelines of the form:
One Hot Encoder -> Resampler -> Model.

In [7]:
def build_pipeline(model, number_resampler = 0):
    """Build a pipeline with the model, a one-hot encoder and a resampler."""
    pipeline = Pipeline([('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore')), 
                               list_resampler[number_resampler],
                               ('model', model)])
    return pipeline

In [12]:
def fit_and_predict(pipeline, columns, number_resampler):
   """Fit the pipeline with the given columns and return the pipeline and a dictionary of data"""
   pipeline.fit(features_train_resampled.loc[:, columns], target_train_resampled)
   target_train_pred = pipeline.predict(features_train_resampled.loc[:,columns])
   target_val_pred = pipeline.predict(features_val.loc[:,columns])
   acc_train = accuracy_score(target_train_resampled, target_train_pred)
   acc_val = accuracy_score(target_val, target_val_pred)
   data = {
   "index": [-1],
    "columns": ['Features', 'Number_features', 'Model_typ', 'Accuracy_train', 'Accuracy_val', 'Model', 'search_space', 'search_res', "resampler"],
    "data": [[columns, len(columns), pipeline.named_steps['model'], acc_train, acc_val, pipeline, {}, None, number_resampler]],
    "index_names": ["index"],
    "column_names": ["Columns"],
    }
   return pipeline, data

## Models


List of models tested, without the artificial neural network.

In [9]:
models = [DecisionTreeClassifier(),
          RandomForestClassifier(random_state=42, n_estimators=5), 
          LogisticRegression(solver='lbfgs'),
          KNeighborsClassifier(),
          LinearSVC(multi_class='crammer_singer')
          ]

Train the models using one feature at a time. 

In [13]:
for model in models:
    for col in all_columns:
        number_resampler = 0
        pipeline = build_pipeline(model, number_resampler)
        pipeline_fitted, data = fit_and_predict(pipeline, [col], number_resampler)
        results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
        results = results.reset_index(drop=True)
    results.to_csv('../data/processed/Comparison_models.csv', index=False)

/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_82670/3627644717.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/qp251849/Desktop/Classification_project/sample-project-c

### Artificial Neural Network


Dictionary which encodes the target values as numbers.

In [15]:
dict_social_grade = {'AB': 4, 'C2': 3, 'C1': 2, 'DE': 1, 'No code required': 0}

Application of the dictionary to the targets.

In [16]:
target_train = target_train.replace(dict_social_grade)
target_train_resampled = target_train_resampled.replace(dict_social_grade)
target_val = target_val.replace(dict_social_grade)

/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_82670/882130119.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target_train = target_train.replace(dict_social_grade)
/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_82670/882130119.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target_train_resampled = target_train_resampled.replace(dict_social_grade)
/var/folders/cf/zk30004x3v91314p4gt9qxlr0000gp/T/ipykernel_82670/882130119.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will

In [17]:
num_classes = 5
target_train_resampled_encoded = tf.keras.utils.to_categorical(target_train_resampled, num_classes=num_classes)
target_val_encoded  = tf.keras.utils.to_categorical(target_val, num_classes=num_classes)

pipeline_an = Pipeline(steps = [('ohe', OneHotEncoder(sparse_output=False, 
                                                      handle_unknown='ignore'))])
for col in all_columns:
    col = [col]
    # Prepare the features for the ANN
    feat_train_transf = pipeline_an.fit_transform(features_train_resampled.loc[:,col])
    feat_val_transf = pipeline_an.transform(features_val.loc[:,col])
    number_resampler = 0

    resampler = list_resampler[number_resampler][1] 
    #resampler = RandomUnderSampler(random_state=42)
    #resampler = RandomOverSampler(random_state=42)
    #feat_train_res, tar_train_res = resampler.fit_resample(feat_train_transf, target_train)
    feat_train_res = feat_train_transf
    tar_train_res = target_train
    model_an = Sequential()
    model_an.add(Dense(1, activation='sigmoid', input_dim=feat_train_res.shape[1]))
    model_an.add(Dense(5, activation = 'relu'))
    model_an.add(Dropout(rate=0.3))
    model_an.add(Dense(num_classes, activation='softmax'))

    model_an.compile(optimizer = "adam", loss = "categorical_crossentropy", 
                     metrics=['accuracy'])  # compile the model

    early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 2)
    hist = model_an.fit(feat_train_res, 
                    target_train_resampled_encoded, 
                    validation_data=(feat_val_transf, target_val_encoded),
                    epochs=200, 
                    batch_size=32,
                    callbacks=[early_stop]
                   )
    test_loss, acc_val = model_an.evaluate(pipeline_an.transform(features_val.loc[:,col]),
                                       target_val_encoded, verbose=2)
    test_loss, acc_train = model_an.evaluate(pipeline_an.transform(features_train_resampled.loc[:,col]),
                                       target_train_resampled_encoded, verbose=2)
    data = {
   "index": [-1],
    "columns": ['Features', 'Number_features', 'Model_typ', 'Accuracy_train', 'Accuracy_val', 'Model', 'search_space', 'search_res', "resampler"],
    "data": [[col, len(col), 'ANN', acc_train, acc_val, model_an, {}, None, number_resampler]],
    "index_names": ["index"],
    "column_names": ["Columns"],
    }
    results = pd.concat([results, pd.DataFrame.from_dict(data, orient="tight")])
    results = results.reset_index(drop=True)
results.to_csv('../data/processed/Comparison_models.csv', index=False)
    

Epoch 1/200


/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2520 - loss: 1.5923 - val_accuracy: 0.2789 - val_loss: 1.5801
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5785 - val_accuracy: 0.2789 - val_loss: 1.5766
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5764 - val_accuracy: 0.2789 - val_loss: 1.5758
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5754 - val_accuracy: 0.2789 - val_loss: 1.5756
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5744 - val_accuracy: 0.2789 - val_loss: 1.5755
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5740 - val_accuracy: 0.2789 - val_loss: 1.5754
3205/3205 - 1s - 265us/step - accuracy: 0.2789 - loss: 1.5754
313/313 - 0s - 305us/step - accuracy: 0.2829 - loss: 1.5736
Epoch 1/200


/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2443 - loss: 1.5989 - val_accuracy: 0.2789 - val_loss: 1.5779
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2790 - loss: 1.5787 - val_accuracy: 0.2789 - val_loss: 1.5751
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5755 - val_accuracy: 0.2789 - val_loss: 1.5747
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5756 - val_accuracy: 0.2789 - val_loss: 1.5747
3205/3205 - 1s - 260us/step - accuracy: 0.2789 - loss: 1.5747
313/313 - 0s - 309us/step - accuracy: 0.2829 - loss: 1.5729
Epoch 1/200


...

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2525 - loss: 1.5872 - val_accuracy: 0.2789 - val_loss: 1.5713
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2829 - loss: 1.5585 - val_accuracy: 0.2789 - val_loss: 1.5368
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2851 - loss: 1.5025 - val_accuracy: 0.3844 - val_loss: 1.4511
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3850 - loss: 1.4286 - val_accuracy: 0.3972 - val_loss: 1.3743
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4007 - loss: 1.3816 - val_accuracy: 0.4131 - val_loss: 1.3256
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4001 - loss: 1.3590 - val_accuracy: 0.4131 - val_loss: 1.2964
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4002 - loss: 1.3416 - val_accuracy: 0.4131 - val_loss: 1.2755
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4011 - loss: 1.3277 - val_accuracy: 0.4131

/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2780 - loss: 1.5846 - val_accuracy: 0.2789 - val_loss: 1.5769
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2812 - loss: 1.5761 - val_accuracy: 0.2789 - val_loss: 1.5725
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2800 - loss: 1.5709 - val_accuracy: 0.2789 - val_loss: 1.5628
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2840 - loss: 1.5519 - val_accuracy: 0.2789 - val_loss: 1.5250
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3715 - loss: 1.4857 - val_accuracy: 0.4544 - val_loss: 1.4148
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4191 - loss: 1.3897 - val_accuracy: 0.4544 - val_loss: 1.3077
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4274 - loss: 1.3288 - val_accuracy: 0.4544 - val_loss: 1.2673
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4312 - loss: 1.3161 - val_accuracy: 0.4544

/Users/qp251849/Desktop/Classification_project/sample-project-classification/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2645 - loss: 1.5984 - val_accuracy: 0.2789 - val_loss: 1.5785
Epoch 2/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2927 - loss: 1.5717 - val_accuracy: 0.2789 - val_loss: 1.5647
Epoch 3/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2984 - loss: 1.5599 - val_accuracy: 0.2789 - val_loss: 1.5506
Epoch 4/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3406 - loss: 1.5460 - val_accuracy: 0.2789 - val_loss: 1.5310
Epoch 5/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3454 - loss: 1.5286 - val_accuracy: 0.3805 - val_loss: 1.5124
Epoch 6/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3421 - loss: 1.5235 - val_accuracy: 0.3805 - val_loss: 1.5001
Epoch 7/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3403 - loss: 1.5178 - val_accuracy: 0.3805 - val_loss: 1.4932
Epoch 8/200
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3462 - loss: 1.5142 - val_accuracy: 0.3805